In [1]:
! pip3 install -q towhee pymilvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.3 MB/s eta 0:00:00


In [2]:
! wget -q https://github.com/towhee-io/examples/releases/download/data/New_Medium_Data.csv

In [ ]:
import pandas as pd
#kaggle datasets download -d shiyu22chen/cleaned-medium-articles-dataset
df = pd.read_csv('New_Medium_Data.csv', 
                 converters={'title_vector': lambda x: eval(x)})
df.head()

In [4]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

In [6]:
connections.connect(
    "default",
    host = 'in01-7790779079.com',
    user = '080',
    port = '88080',
    password = '0808',
    secure = True
)

In [7]:
def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
            FieldSchema(name="id", 
                        dtype=DataType.INT64, 
                        is_primary=True, 
                        auto_id=False),
            FieldSchema(name="title", 
                        dtype=DataType.VARCHAR, 
                        max_length=500),   
            FieldSchema(name="title_vector", 
                        dtype=DataType.FLOAT_VECTOR, 
                        dim=dim),
            FieldSchema(name="link", 
                        dtype=DataType.VARCHAR, 
                        max_length=500),
            FieldSchema(name="reading_time", 
                        dtype=DataType.INT64),
            FieldSchema(name="publication", 
                        dtype=DataType.VARCHAR, 
                        max_length=500),
            FieldSchema(name="claps", 
                        dtype=DataType.INT64),
            FieldSchema(name="responses", 
                        dtype=DataType.INT64)
    ]
    
    schema = CollectionSchema(fields=fields, 
                              description='search text')
    
    collection = Collection(name=collection_name, 
                            schema=schema)
    
    index_params = {
        'metric_type': "L2",
        'index_type': "AUTOINDEX",
        'params': {"nlist": 2048}
    }
    collection.create_index(field_name='title_vector', 
                            index_params=index_params)
    return collection

In [16]:
collection = create_milvus_collection('search_article_in_medium', 
                                      768)

In [10]:
from towhee.dc2 import ops, pipe, DataCollection

insert_pipe = (pipe.input('df')
                   .flat_map('df', 'data', lambda df: df.values.tolist())
                   .map('data', 'res', ops.ann_insert.milvus_client(host = 'in01-8080086068.com',
                                                                    user = '',
                                                                    port = '',
                                                                    password = '',
                                                                    collection_name='search_article_in_medium'))
                   .output('res')
)

In [20]:
collection.load()

In [ ]:
insert_pipe(df)

In [22]:
collection.num_entities

2002

In [23]:
import numpy as np

search_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score'), ops.ann_search.milvus_client(host = 'in01-8z8090909.com',
                                                                                  user = '',
                                                                                  port = '',
                                                                                  password = '',
                                                                                  collection_name='search_article_in_medium'))  
                    .output('query', 'id', 'score')
               )

Cloning the repo: text-embedding/dpr... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: text-embedding/dpr.


Cloning the repo: ann-search/milvus-client... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: ann-search/milvus-client.


In [24]:
res = search_pipe('funny python demo')
DataCollection(res).show()

query,id,score
funny python demo,1342,0.43680664896965027
funny python demo,1832,0.45723867416381836
funny python demo,1752,0.4645399749279022
funny python demo,1172,0.4901197552680969
funny python demo,1667,0.5019433498382568


In [27]:
search_pipe1 = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'title'), ops.ann_search.milvus_client(host = 'in01-8888zz8.zillizcloud.com',
                                                                                  user = '',
                                                                                  port = '',
                                                                                  password = '',
                                                                                   collection_name='search_article_in_medium',
                                                                                   output_fields=['title']))  
                    .output('query', 'id', 'score', 'title')
               )

In [28]:
res = search_pipe1('funny python demo')
DataCollection(res).show()

query,id,score,title
funny python demo,1342,0.43680664896965027,How to Design Professional Venn Diagrams in Python
funny python demo,1832,0.45723867416381836,How to mock AWS services for rapid local development.
funny python demo,1752,0.4645399749279022,Custom neural networks in Keras: a street fighter’s guide to build a graphCNN
funny python demo,1172,0.4901197552680969,OOP Practice in Python: Bike Share Simulator
funny python demo,1667,0.5019433498382568,Python — How to measure thread execution time in multithreaded application?


In [29]:
# milvus search with multi outpt fields
search_pipe2 = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'title', 'link', 'reading_time', 'publication', 'claps', 'responses'), 
                                       ops.ann_search.milvus_client(host = 'in01-8777.8866.vectordb.8z8.com',
                                                                    user = '',
                                                                    port = '',
                                                                    password = '',
                                                                    collection_name='search_article_in_medium',
                                                                    output_fields=['title', 'link', 'reading_time', 'publication', 'claps', 'responses'], 
                                                                    limit=5))  
                    .output('query', 'id', 'score', 'title', 'link', 'reading_time', 'publication', 'claps', 'responses')
               )

res = search_pipe2('funny python demo')
DataCollection(res).show()

query,id,score,title,link,reading_time,publication,claps,responses
funny python demo,1342,0.43680664896965027,How to Design Professional Venn Diagrams in Python,https://towardsdatascience.com/how-to-design-professional-venn-diagrams-in-python-693c9ed2c288,6,Towards Data Science,97,1
funny python demo,1832,0.45723867416381836,How to mock AWS services for rapid local development.,https://medium.com/swlh/how-to-mock-aws-services-for-rapid-local-development-3d07581ffc3a,3,The Startup,84,0
funny python demo,1752,0.4645399749279022,Custom neural networks in Keras: a street fighter’s guide to build a graphCNN,https://towardsdatascience.com/custom-neural-networks-in-keras-a-street-fighters-guide-to-build-a-graphcnn-e91f6b05f12e,7,Towards Data Science,55,0
funny python demo,1172,0.4901197552680969,OOP Practice in Python: Bike Share Simulator,https://medium.com/swlh/oop-practice-in-python-bike-share-simulator-4717bd12b365,4,The Startup,152,0
funny python demo,1667,0.5019433498382568,Python — How to measure thread execution time in multithreaded application?,https://medium.com/swlh/python-how-to-measure-thread-execution-time-in-multithreaded-application-f4b2e2112091,6,The Startup,55,0


In [30]:
search_pipe3 = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score', 'title', 'link', 'reading_time', 'publication', 'claps', 'responses'), 
                                       ops.ann_search.milvus_client(host = 'in01-877878787.com',
                                                                    user = '',
                                                                    port = '',
                                                                    password = '',
                                                                    collection_name='search_article_in_medium',
                                                                    expr='title like "Python%"',
                                                                    output_fields=['title', 'link', 'reading_time', 'publication', 'claps', 'responses'], 
                                                                    limit=5))  
                    .output('query', 'id', 'score', 'title', 'link', 'reading_time', 'publication', 'claps', 'responses')
               )

res = search_pipe3('funny python demo')
DataCollection(res).show()

query,id,score,title,link,reading_time,publication,claps,responses
funny python demo,1667,0.5019433498382568,Python — How to measure thread execution time in multithreaded application?,https://medium.com/swlh/python-how-to-measure-thread-execution-time-in-multithreaded-application-f4b2e2112091,6,The Startup,55,0
funny python demo,1298,0.5166990756988525,Python Testing with a mock database (SQL),https://medium.com/swlh/python-testing-with-a-mock-database-sql-68f676562461,4,The Startup,51,0
funny python demo,620,0.5193841457366943,Python Function-Tools Tricks!,https://towardsdatascience.com/python-function-tools-tricks-127a0eea87ed,2,Towards Data Science,30,1
funny python demo,1315,0.5281139612197876,Python Virtual Environments,https://medium.com/swlh/python-virtual-environments-221c39943bab,2,The Startup,80,0
funny python demo,4,0.5284628868103027,Python NLP Tutorial: Information Extraction and Knowledge Graphs,https://medium.com/swlh/python-nlp-tutorial-information-extraction-and-knowledge-graphs-43a2a4c4556c,7,The Startup,163,0


In [31]:
collection.query(
  expr = 'claps > 3000 && reading_time < 15 && publication like "Towards Data Science%"', 
  output_fields = ['id', 'title', 'link', 'reading_time', 'publication', 'claps', 'responses'],
  consistency_level='Strong'
)

[{'title': 'I Thought I Was Mastering Python Until I Discovered These Tricks',
  'link': 'https://towardsdatascience.com/i-thought-i-was-mastering-python-until-i-discovered-these-tricks-e40d9c71f4e2',
  'reading_time': 9,
  'publication': 'Towards Data Science',
  'claps': 5200,
  'responses': 17,
  'id': 913},
 {'title': 'Do Not Use “+” to Join Strings in Python',
  'link': 'https://towardsdatascience.com/do-not-use-to-join-strings-in-python-f89908307273',
  'reading_time': 5,
  'publication': 'Towards Data Science',
  'claps': 4200,
  'responses': 28,
  'id': 1195}]